In [ ]:
# ============================================================
# WingMate AI — Context-Aware Co-Vis Recommender (Clean Final)
# ============================================================
# What you get:
#   - Robust CSV reader (handles stray commas in ORDERS column)
#   - Context co-visitation (global + channel + subchannel + occasion + store + customer type)
#   - Tuned blend weights + optional MMR diversity
#   - Two outputs: MAX (submit) and TUNED (demo)
#   - Strict LOO & Temporal holdout evaluations (judge-friendly)
#   - Optional LightGBM reranker (compact features)
#
# Usage (script):
#   python unravel_final.py \
#       --order_path /content/order_data.csv \
#       --test_path  /content/test_data_question.csv \
#       --out_dir    /content \
#       --use_reranker false \
#       --do_eval true \
#       --do_temporal true
#
# Or: run all cells in notebook order.
# ------------------------------------------------------------

import os, gc, math, random, re, json, csv, time, argparse
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from tqdm import tqdm

# -------------------
# 1) Paths & Repro
# -------------------
def set_seed(s=42):
    random.seed(s); np.random.seed(s)
set_seed(42)

# Defaults (override via CLI)
ORDER_PATH = "/content/order_data.csv"
TEST_PATH  = "/content/test_data_question.csv"
OUT_DIR    = "/content"

# Derived outputs
def out_paths(out_dir):
    return {
        "OUT_MAX":   os.path.join(out_dir, "Recommendation_Output_MAX.xlsx"),
        "OUT_TUNED": os.path.join(out_dir, "Recommendation_Output_TUNED.xlsx"),
        "REASONS":   os.path.join(out_dir, "Recommendation_With_Reasons.csv"),
        "MET_JSON":  os.path.join(out_dir, "metrics.json"),
        "MET_CSV":   os.path.join(out_dir, "metrics.csv"),
    }

# -----------------------------
# 2) Robust reader & utilities
# -----------------------------
EXPECTED_COLS = ("CUSTOMER_ID","ORDER_ID",
                 "ORDER_CHANNEL_NAME","ORDER_SUBCHANNEL_NAME",
                 "ORDER_OCCASION_NAME","STORE_NUMBER",
                 "CUSTOMER_TYPE","ORDERS")

def row_iter(path, expected_cols=EXPECTED_COLS):
    """
    Robust iterator for large/messy CSVs.
    Splits only on the first 7 commas; keeps the rest of the line as last field (ORDERS).
    """
    exp_n = len(expected_cols)  # 8
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        _ = f.readline()  # header line (ignored)
        for line in f:
            line = line.rstrip("\n")
            if not line:
                continue
            parts = line.split(",", exp_n - 1)  # keep extra commas in last field
            if len(parts) < exp_n:
                continue
            yield dict(zip(expected_cols, parts[:exp_n]))

def normalize_text(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def extract_items(order_text) -> list:
    if order_text is None: return []
    t = str(order_text).strip().strip('"').strip("'")
    if not t: return []
    # Accept delims: | , ;
    parts = re.split(r"[|,;]\s*", t)
    items = []
    seen = set()
    for p in parts:
        pp = normalize_text(p)
        if not pp: continue
        if pp.lower() in {"na","none","null","nan"}: continue
        if pp not in seen:
            seen.add(pp); items.append(pp)
    return items

def safe_get(row, key, default=""):
    return normalize_text(row.get(key, default))

def first_token(s: str) -> str:
    s = normalize_text(s).upper()
    m = re.match(r"[A-Z0-9]+", s)
    return m.group(0) if m else s[:8]

# --------------------------------------------
# 3) Build co-visitation maps + popularity
# --------------------------------------------
def build_covis_maps(order_path):
    covis_global      = defaultdict(Counter)
    covis_by_channel  = defaultdict(lambda: defaultdict(Counter))
    covis_by_subch    = defaultdict(lambda: defaultdict(Counter))
    covis_by_occ      = defaultdict(lambda: defaultdict(Counter))
    covis_by_store    = defaultdict(lambda: defaultdict(Counter))
    covis_by_custtype = defaultdict(lambda: defaultdict(Counter))
    pop               = Counter()

    t0=time.time()
    rows_parsed=0; orders_with_items=0; uniq_items=set()

    for row in tqdm(row_iter(order_path), desc="Building co-vis maps"):
        items = extract_items(row.get("ORDERS", ""))
        if not items:
            continue
        orders_with_items += 1; rows_parsed += 1

        # popularity (unique per order)
        for it in set(items):
            pop[it] += 1; uniq_items.add(it)

        uniq = list(set(items))
        n = len(uniq)
        if n > 1:
            ch = row.get("ORDER_CHANNEL_NAME","")
            sc = row.get("ORDER_SUBCHANNEL_NAME","")
            oc = row.get("ORDER_OCCASION_NAME","")
            st = row.get("STORE_NUMBER","")
            ct = row.get("CUSTOMER_TYPE","")
            for i in range(n):
                a = uniq[i]
                for j in range(i+1, n):
                    b = uniq[j]
                    covis_global[a][b]+=1; covis_global[b][a]+=1
                    covis_by_channel[ch][a][b]+=1;   covis_by_channel[ch][b][a]+=1
                    covis_by_subch[sc][a][b]+=1;     covis_by_subch[sc][b][a]+=1
                    covis_by_occ[oc][a][b]+=1;       covis_by_occ[oc][b][a]+=1
                    covis_by_store[st][a][b]+=1;     covis_by_store[st][b][a]+=1
                    covis_by_custtype[ct][a][b]+=1;  covis_by_custtype[ct][b][a]+=1

        if rows_parsed % 200_000 == 0:
            print(f"Processed {rows_parsed:,} rows...")

    t1=time.time()
    print(f"Rows parsed: {rows_parsed:,} | Orders with items: {orders_with_items:,} | Unique items: {len(pop)}")
    print(f"Build time: {round((t1-t0)/60, 2)} min")
    return covis_global, covis_by_channel, covis_by_subch, covis_by_occ, covis_by_store, covis_by_custtype, pop

def normalize_covis(cmap: dict, pop: Counter, power=0.5, keep_top=500):
    out = {}
    for a, neigh in cmap.items():
        pa = max(pop[a], 1)
        denom_a = pa**power
        d = {}
        for b, c in neigh.items():
            pb = max(pop[b], 1)
            d[b] = float(c) / (denom_a * (pb**power))
        out[a] = dict(Counter(d).most_common(keep_top))
    return out

def norm_nested(ctx_map, pop):
    out = {}
    for key, cmap in ctx_map.items():
        out[key] = normalize_covis(cmap, pop, power=0.5)
    return out

# -------------------------------------
# 4) Scoring blend + MMR (diversity)
# -------------------------------------
# Tuned weights (from your good runs)
W        = (0.51, 0.24, 0.05, 0.11, 0.17, 0.10)  # (global, channel, subchannel, occasion, store, customer_type)
LAM      = 0.93
BACKOFF_ALPHA = 0.15
CAND_POOL     = 120
BACKFILL_POOL = 800

def blended_scores(cart, row, covis_global, covis_by_channel, covis_by_subch, covis_by_occ, covis_by_store, covis_by_custtype, pop, w=W):
    scores = Counter()
    def add_from(map_, weight):
        if weight <= 0 or not map_: return
        for it in cart:
            for c, v in map_.get(it, {}).items():
                scores[c] += weight * v

    ch = str(row.get("ORDER_CHANNEL_NAME", ""))
    sc = str(row.get("ORDER_SUBCHANNEL_NAME", ""))
    oc = str(row.get("ORDER_OCCASION_NAME", ""))
    st = str(row.get("STORE_NUMBER", ""))
    ct = str(row.get("CUSTOMER_TYPE", ""))

    add_from(covis_global, w[0])
    if ch in covis_by_channel:  add_from(covis_by_channel[ch],  w[1])
    if sc in covis_by_subch:    add_from(covis_by_subch[sc],    w[2])
    if oc in covis_by_occ:      add_from(covis_by_occ[oc],      w[3])
    if st in covis_by_store:    add_from(covis_by_store[st],    w[4])
    if ct in covis_by_custtype: add_from(covis_by_custtype[ct], w[5])

    for it in cart:
        scores.pop(it, None)

    # sparse backoff
    if len(scores) < 5:
        for it in cart:
            for c, v in covis_global.get(it, {}).items():
                scores[c] += BACKOFF_ALPHA * v
        for it in cart: scores.pop(it, None)

    return scores

def mmr_select(cands, k=3, lam=LAM):
    sel=[]
    def sim(a,b):
        ta=set(first_token(a).split()) | set(a.upper().split())
        tb=set(first_token(b).split()) | set(b.upper().split())
        inter=len(ta & tb); denom=math.sqrt(len(ta)*len(tb)) or 1.0
        return inter/denom
    work=cands[:]
    while work and len(sel)<k:
        best=None; bestv=-1e9
        for idx,c in enumerate(work):
            rel=1.0/(1.0+idx)
            div=0.0 if not sel else max(sim(c,s) for s in sel)
            v=lam*rel - (1-lam)*div
            if v>bestv: best,bestv=c,v
        sel.append(best); work.remove(best)
    return sel

def recommend_ctx(cart, row, maps, global_top, pop, k=3, use_mmr=False):
    (covis_global, covis_by_channel, covis_by_subch, covis_by_occ, covis_by_store, covis_by_custtype) = maps
    scores = blended_scores(cart, row, covis_global, covis_by_channel, covis_by_subch, covis_by_occ, covis_by_store, covis_by_custtype, pop)
    cands = [c for c,_ in scores.most_common(BACKFILL_POOL) if c not in cart][:CAND_POOL]
    # backfill with popularity
    for g in global_top:
        if len(cands) >= CAND_POOL: break
        if g not in cart and g not in cands: cands.append(g)
    final = mmr_select(cands, k=k) if use_mmr else cands[:k]
    return (final + ["","",""])[:k]

# ---------------------------------
# 5) Inference + output writers
# ---------------------------------
def write_excel(test_path, out_path, maps, global_top, pop, use_mmr=False, reasons_csv=None):
    df = pd.read_csv(test_path, dtype=str, keep_default_na=False)
    item_cols = [c for c in df.columns if c.upper().startswith("ITEM")]
    p1,p2,p3 = [],[],[]
    reasons_rows=[]
    for _, r in tqdm(df.iterrows(), total=len(df), desc=("Predict MMR" if use_mmr else "Predict Max")):
        cart = [normalize_text(r[c]) for c in item_cols if r.get(c,"")]
        recs = recommend_ctx(cart, r, maps, global_top, pop, k=3, use_mmr=use_mmr)
        p1.append(recs[0]); p2.append(recs[1]); p3.append(recs[2])
        if reasons_csv is not None:
            reasons_rows.append({
                "ORDER_ID": r.get("ORDER_ID",""),
                "CART": " | ".join(cart[:8]),
                "REC1": recs[0],
                "REC2": recs[1],
                "REC3": recs[2],
            })
    out = df.copy()
    out["RECOMMENDATION 1"] = p1
    out["RECOMMENDATION 2"] = p2
    out["RECOMMENDATION 3"] = p3
    cols = ["CUSTOMER_ID","ORDER_ID"] + item_cols + ["RECOMMENDATION 1","RECOMMENDATION 2","RECOMMENDATION 3"]
    # Write Excel (fallback to CSV if engine missing)
    try:
        out[cols].to_excel(out_path, index=False)
    except Exception:
        out_csv = out_path.replace(".xlsx", ".csv")
        out[cols].to_csv(out_csv, index=False)
        print(f"[INFO] openpyxl not found; wrote CSV instead: {out_csv}")
    if reasons_csv is not None:
        pd.DataFrame(reasons_rows).to_csv(reasons_csv, index=False)
    print(f"Wrote: {out_path}")

# ---------------------------------
# 6) Offline evaluations (strict & temporal)
# ---------------------------------
def eval_strict_loo(order_path, maps, global_top, pop, n_eval=5000):
    hits1=hits2=hits3=map3=ndcg3=0.0; seen=0
    for row in row_iter(order_path):
        items = extract_items(row.get("ORDERS",""))
        uniq = list(dict.fromkeys(items))  # preserve order, drop dups
        if len(uniq) < 2: continue
        target = uniq[-1]
        cart = [x for x in uniq if x != target]
        if not cart: continue
        preds = recommend_ctx(cart, row, maps, global_top, pop, k=3, use_mmr=False)
        if len(preds)>=1 and preds[0]==target: hits1+=1
        if target in preds[:2]: hits2+=1
        if target in preds[:3]: hits3+=1
        if target in preds[:3]:
            rank = preds.index(target)+1
            map3 += 1.0/rank
            ndcg3+= 1.0/math.log2(rank+1)
        seen += 1
        if seen >= n_eval: break
    if seen==0:
        return {"num_eval_orders":0,"Recall@1":0,"Recall@2":0,"Recall@3":0,"MAP@3":0,"NDCG@3":0}
    return {"num_eval_orders": seen,
            "Recall@1": round(hits1/seen,4),
            "Recall@2": round(hits2/seen,4),
            "Recall@3": round(hits3/seen,4),
            "MAP@3": round(map3/seen,4),
            "NDCG@3": round(ndcg3/seen,4)}

def count_rows_fast(path):
    return max(0, sum(1 for _ in open(path, "r", encoding="utf-8", errors="replace")) - 1)

def build_covis_upto(order_path, limit_rows):
    cg      = defaultdict(Counter)
    by_ch   = defaultdict(lambda: defaultdict(Counter))
    by_sc   = defaultdict(lambda: defaultdict(Counter))
    by_oc   = defaultdict(lambda: defaultdict(Counter))
    by_st   = defaultdict(lambda: defaultdict(Counter))
    by_ct   = defaultdict(lambda: defaultdict(Counter))
    pop     = Counter()
    i=0
    for row in row_iter(order_path):
        if i >= limit_rows: break
        items = extract_items(row.get("ORDERS",""))
        if not items:
            i+=1; continue
        for it in set(items): pop[it] += 1
        uniq = list(set(items))
        n = len(uniq)
        if n > 1:
            ch=row.get("ORDER_CHANNEL_NAME",""); sc=row.get("ORDER_SUBCHANNEL_NAME","")
            oc=row.get("ORDER_OCCASION_NAME",""); st=row.get("STORE_NUMBER","")
            ct=row.get("CUSTOMER_TYPE","")
            for a_i in range(n):
                a = uniq[a_i]
                for b_i in range(a_i+1, n):
                    b = uniq[b_i]
                    cg[a][b]+=1; cg[b][a]+=1
                    by_ch[ch][a][b]+=1;  by_ch[ch][b][a]+=1
                    by_sc[sc][a][b]+=1;  by_sc[sc][b][a]+=1
                    by_oc[oc][a][b]+=1;  by_oc[oc][b][a]+=1
                    by_st[st][a][b]+=1;  by_st[st][b][a]+=1
                    by_ct[ct][a][b]+=1;  by_ct[ct][b][a]+=1
        i += 1
    global_top = [it for it,_ in pop.most_common(1000)]
    return cg, by_ch, by_sc, by_oc, by_st, by_ct, pop, global_top

def normalize_all(cg, by_ch, by_sc, by_oc, by_st, by_ct, pop):
    cg = normalize_covis(cg, pop, power=0.5)
    by_ch = norm_nested(by_ch, pop)
    by_sc = norm_nested(by_sc, pop)
    by_oc = norm_nested(by_oc, pop)
    by_st = norm_nested(by_st, pop)
    by_ct = norm_nested(by_ct, pop)
    return cg, by_ch, by_sc, by_oc, by_st, by_ct

def temporal_eval(order_path, frac_train=0.90, max_eval=8000):
    total = count_rows_fast(order_path)
    cut = int(total * frac_train)
    print(f"[Temporal] total={total:,} train_first={cut:,} test_last={total-cut:,}")
    cg, by_ch, by_sc, by_oc, by_st, by_ct, pop, gtop = build_covis_upto(order_path, cut)
    cg, by_ch, by_sc, by_oc, by_st, by_ct = normalize_all(cg, by_ch, by_sc, by_oc, by_st, by_ct, pop)
    maps = (cg, by_ch, by_sc, by_oc, by_st, by_ct)
    hits1=hits2=hits3=map3=ndcg3=0.0; seen=0; i=0
    for row in row_iter(order_path):
        i+=1
        if i<=cut: continue
        items = extract_items(row.get("ORDERS",""))
        uniq = list(dict.fromkeys(items))
        if len(uniq)<2: continue
        target=uniq[-1]
        cart = [x for x in uniq if x!=target]
        if not cart: continue
        preds = recommend_ctx(cart, row, maps, gtop, pop, k=3, use_mmr=False)
        if len(preds)>=1 and preds[0]==target: hits1+=1
        if target in preds[:2]: hits2+=1
        if target in preds[:3]: hits3+=1
        if target in preds[:3]:
            rank = preds.index(target)+1
            map3 += 1.0/rank
            ndcg3+= 1.0/math.log2(rank+1)
        seen += 1
        if seen >= max_eval: break
    if seen==0:
        return {"num_eval_orders":0,"Recall@1":0,"Recall@2":0,"Recall@3":0,"MAP@3":0,"NDCG@3":0}
    return {"num_eval_orders": seen,
            "Recall@1": round(hits1/seen,4),
            "Recall@2": round(hits2/seen,4),
            "Recall@3": round(hits3/seen,4),
            "MAP@3": round(map3/seen,4),
            "NDCG@3": round(ndcg3/seen,4)}

# ---------------------------------
# 7) Optional: LightGBM reranker
# ---------------------------------
def train_reranker(order_path, maps, global_top, pop, n_orders_train=300_000, n_orders_valid=80_000, cand_top=80):
    import lightgbm as lgb
    from lightgbm import LGBMRanker
    X_tr=[]; y_tr=[]; g_tr=[]
    X_va=[]; y_va=[]; g_va=[]
    cnt=0
    def make_feats(cart, row, cand):
        f0 = pop.get(cand, 0)
        f1 = sum(maps[0].get(x,{}).get(cand,0) for x in cart)  # global co-vis sum
        f2 = len(cart)
        f3 = int(cand in global_top[:50])
        return [f0, f1, f2, f3]
    for row in row_iter(order_path):
        items = extract_items(row.get("ORDERS",""))
        if len(items) < 2: continue
        target = items[-1]; cart = items[:-1]
        scores = blended_scores(cart, row, *maps, pop)
        cands = [c for c,_ in scores.most_common(cand_top) if c not in cart]
        if not cands: continue
        X = [make_feats(cart, row, c) for c in cands]
        Y = [1 if c==target else 0 for c in cands]
        if cnt < n_orders_train:
            X_tr.extend(X); y_tr.extend(Y); g_tr.append(len(X))
        elif cnt < n_orders_train+n_orders_valid:
            X_va.extend(X); y_va.extend(Y); g_va.append(len(X))
        else:
            break
        cnt+=1
        if cnt % 200_000 == 0:
            print(f"Built {cnt:,} training orders...")
    ranker = LGBMRanker(
        n_estimators=200, learning_rate=0.08, max_depth=-1,
        subsample=0.85, colsample_bytree=0.85, objective="lambdarank",
        random_state=42
    )
    ranker.fit(np.array(X_tr, dtype=np.float32), np.array(y_tr), group=g_tr,
               eval_set=[(np.array(X_va, dtype=np.float32), np.array(y_va))], eval_group=[g_va])
    return ranker

def predict_with_reranker(cart, row, maps, global_top, pop, ranker, k=3, cand_top=120):
    scores = blended_scores(cart, row, *maps, pop)
    cands = [c for c,_ in scores.most_common(cand_top) if c not in cart]
    if not cands:
        return (global_top[:k] + ["","",""])[:k]
    # same 4 lightweight features as training
    def make_feats(cart, row, cand):
        f0 = pop.get(cand, 0)
        f1 = sum(maps[0].get(x,{}).get(cand,0) for x in cart)
        f2 = len(cart)
        f3 = int(cand in global_top[:50])
        return [f0, f1, f2, f3]
    X = np.array([make_feats(cart, row, c) for c in cands], dtype=np.float32)
    preds = ranker.predict(X)
    order = np.argsort(-preds)
    ranked = [cands[i] for i in order[:k]]
    return (ranked + ["","",""])[:k]

# ---------------------------------
# 8) Main
# ---------------------------------
def main(order_path=ORDER_PATH, test_path=TEST_PATH, out_dir=OUT_DIR,
         use_reranker=False, do_eval=True, do_temporal=True):
    os.makedirs(out_dir, exist_ok=True)
    paths = out_paths(out_dir)

    # Build & normalize
    (cg, by_ch, by_sc, by_oc, by_st, by_ct, pop) = build_covis_maps(order_path)
    cg      = normalize_covis(cg, pop, power=0.5)
    by_ch   = norm_nested(by_ch, pop)
    by_sc   = norm_nested(by_sc, pop)
    by_oc   = norm_nested(by_oc, pop)
    by_st   = norm_nested(by_st, pop)
    by_ct   = norm_nested(by_ct, pop)
    maps    = (cg, by_ch, by_sc, by_oc, by_st, by_ct)
    global_top = [it for it,_ in pop.most_common(2000)]

    # Submissions
    write_excel(test_path, paths["OUT_MAX"],   maps, global_top, pop, use_mmr=False, reasons_csv=paths["REASONS"])
    write_excel(test_path, paths["OUT_TUNED"], maps, global_top, pop, use_mmr=True)

    # Optional reranker (if you want a 3rd sheet for your deck)
    if use_reranker:
        print("Training LightGBM reranker (optional)...")
        ranker = train_reranker(order_path, maps, global_top, pop)
        # produce an LTR file as well (for storytelling)
        df = pd.read_csv(test_path, dtype=str, keep_default_na=False)
        item_cols = [c for c in df.columns if c.upper().startswith("ITEM")]
        p1,p2,p3=[],[],[]
        for _, r in tqdm(df.iterrows(), total=len(df), desc="Predict LTR"):
            cart = [normalize_text(r[c]) for c in item_cols if r.get(c,"")]
            recs = predict_with_reranker(cart, r, maps, global_top, pop, ranker, k=3)
            p1.append(recs[0]); p2.append(recs[1]); p3.append(recs[2])
        out = df.copy()
        out["RECOMMENDATION 1"]=p1; out["RECOMMENDATION 2"]=p2; out["RECOMMENDATION 3"]=p3
        out.to_excel(os.path.join(out_dir, "Recommendation_Output_LTR.xlsx"), index=False)
        print("Wrote:", os.path.join(out_dir, "Recommendation_Output_LTR.xlsx"))

    # Evaluations for deck (judge-friendly)
    metrics = {}
    if do_eval:
        print("Evaluating (strict LOO sample)...")
        m_strict = eval_strict_loo(order_path, maps, global_top, pop, n_eval=5000)
        print("Strict LOO metrics:", m_strict); metrics["strict_looo"] = m_strict
    if do_temporal:
        print("Evaluating (temporal holdout sample)...")
        m_temp = temporal_eval(order_path, frac_train=0.90, max_eval=8000)
        print("Temporal metrics:", m_temp); metrics["temporal"] = m_temp

    # Save metrics
    if metrics:
        with open(paths["MET_JSON"], "w") as f:
            json.dump(metrics, f, indent=2)
        with open(paths["MET_CSV"], "w") as f:
            f.write("Eval,Num,Recall@1,Recall@2,Recall@3,MAP@3,NDCG@3\n")
            if "strict_looo" in metrics:
                m=metrics["strict_looo"]
                f.write(f"Strict LOO,{m['num_eval_orders']},{m['Recall@1']},{m['Recall@2']},{m['Recall@3']},{m['MAP@3']},{m['NDCG@3']}\n")
            if "temporal" in metrics:
                m=metrics["temporal"]
                f.write(f"Temporal,{m['num_eval_orders']},{m['Recall@1']},{m['Recall@2']},{m['Recall@3']},{m['MAP@3']},{m['NDCG@3']}\n")
        print("Saved metrics:", paths["MET_JSON"], "and", paths["MET_CSV"])

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--order_path", type=str, default=ORDER_PATH)
    parser.add_argument("--test_path",  type=str, default=TEST_PATH)
    parser.add_argument("--out_dir",    type=str, default=OUT_DIR)
    parser.add_argument("--use_reranker", type=lambda x: str(x).lower() in {"1","true","yes","y"}, default=False)
    parser.add_argument("--do_eval",    type=lambda x: str(x).lower() in {"1","true","yes","y"}, default=True)
    parser.add_argument("--do_temporal",type=lambda x: str(x).lower() in {"1","true","yes","y"}, default=True)
    args = parser.parse_args()

    main(order_path=args.order_path,
         test_path=args.test_path,
         out_dir=args.out_dir,
         use_reranker=args.use_reranker,
         do_eval=args.do_eval,
         do_temporal=args.do_temporal)


usage: colab_kernel_launcher.py [-h] [--order_path ORDER_PATH]
                                [--test_path TEST_PATH] [--out_dir OUT_DIR]
                                [--use_reranker USE_RERANKER]
                                [--do_eval DO_EVAL]
                                [--do_temporal DO_TEMPORAL]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-d074196e-a292-47cd-af92-36521897338d.json


SystemExit: 2

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# --- CLI / entrypoint (put this at the very end of your script) ---
import argparse

def build_parser():
    p = argparse.ArgumentParser()
    p.add_argument("--order_path", default="/content/order_data.csv")
    p.add_argument("--test_path",  default="/content/test_data_question.csv")
    p.add_argument("--out_dir",    default="/content")
    p.add_argument("--use_reranker", type=lambda s: s.lower() in {"1","true","yes"}, default=False)
    p.add_argument("--do_eval",      type=lambda s: s.lower() in {"1","true","yes"}, default=True)
    p.add_argument("--do_temporal",  type=lambda s: s.lower() in {"1","true","yes"}, default=False)
    return p

def main_cli():
    parser = build_parser()
    # 👇 swallow notebook/kernel flags like "-f /path/kernel.json"
    args, _unknown = parser.parse_known_args()
    # call your existing main using args
    # main(order_path=args.order_path, test_path=args.test_path, out_dir=args.out_dir,
    #      use_reranker=args.use_reranker, do_eval=args.do_eval, do_temporal=args.do_temporal)
    main(
        order_path=args.order_path,
        test_path=args.test_path,
        out_dir=args.out_dir,
        use_reranker=args.use_reranker,
        do_eval=args.do_eval,
        do_temporal=args.do_temporal
    )

if __name__ == "__main__":
    main_cli()


Building co-vis maps: 200557it [00:54, 3864.48it/s]

Processed 200,000 rows...


Building co-vis maps: 400365it [01:51, 3444.52it/s]

Processed 400,000 rows...


Building co-vis maps: 519662it [02:27, 3531.40it/s]


Rows parsed: 519,662 | Orders with items: 519,662 | Unique items: 3463
Build time: 2.45 min


Predict Max: 100%|██████████| 1000/1000 [00:00<00:00, 6038.34it/s]


Wrote: /content/Recommendation_Output_MAX.xlsx


Predict MMR: 100%|██████████| 1000/1000 [00:02<00:00, 436.45it/s]


Wrote: /content/Recommendation_Output_TUNED.xlsx
Evaluating (strict LOO sample)...
Strict LOO metrics: {'num_eval_orders': 5000, 'Recall@1': 0.8724, 'Recall@2': 0.8738, 'Recall@3': 0.874, 'MAP@3': 0.8732, 'NDCG@3': 0.8734}
Saved metrics: /content/metrics.json and /content/metrics.csv


In [ ]:
import pandas as pd

out_dir = "/content"  # change if needed
max_path   = f"{out_dir}/Recommendation_Output_MAX.xlsx"
tuned_path = f"{out_dir}/Recommendation_Output_TUNED.xlsx"

# 1) Columns exist
req_cols = {"CUSTOMER_ID","ORDER_ID","RECOMMENDATION 1","RECOMMENDATION 2","RECOMMENDATION 3"}
df = pd.read_excel(max_path)
assert req_cols.issubset(df.columns), f"Missing columns: {req_cols - set(df.columns)}"

# 2) Row count matches test set
test_df = pd.read_csv("/content/test_data_question.csv", dtype=str, keep_default_na=False)
assert len(df)==len(test_df), f"Row mismatch: {len(df)} vs test {len(test_df)}"

# 3) No blanks in recs
assert df[["RECOMMENDATION 1","RECOMMENDATION 2","RECOMMENDATION 3"]].notna().all().all()

# 4) No duplicate recs per row
dups = (df["RECOMMENDATION 1"]==df["RECOMMENDATION 2"]) | \
       (df["RECOMMENDATION 1"]==df["RECOMMENDATION 3"]) | \
       (df["RECOMMENDATION 2"]==df["RECOMMENDATION 3"])
assert dups.sum()==0, f"Found {dups.sum()} rows with duplicate recommendations"

print("✅ Output sheet passes all checks.")


✅ Output sheet passes all checks.


In [ ]:
# ==== CONFIG (edit if needed) ====
ORDER_PATH = "/content/order_data.csv"
TEST_PATH  = "/content/test_data_question.csv"
OUT_DIR    = "/content"

import os, re, json, math, datetime, random
from collections import defaultdict, Counter
import numpy as np, pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

os.makedirs(OUT_DIR, exist_ok=True)
random.seed(42); np.random.seed(42)

# -----------------------------
# Helpers (robust row reader + item parsing)
# -----------------------------
EXPECTED_COLS = ("CUSTOMER_ID","ORDER_ID",
                 "ORDER_CHANNEL_NAME","ORDER_SUBCHANNEL_NAME",
                 "ORDER_OCCASION_NAME","STORE_NUMBER",
                 "CUSTOMER_TYPE","ORDERS")

def normalize_text(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def split_items(s: str):
    if s is None: return []
    t = str(s).strip().strip('"').strip("'")
    if not t: return []
    if t.startswith("[") and t.endswith("]"):
        try:
            arr = json.loads(t)
            return [normalize_text(x) for x in arr if str(x).strip()]
        except Exception:
            pass
    for sep in ["|",";","||","\t",","]:
        if sep in t:
            return [normalize_text(x) for x in t.split(sep) if x.strip()]
    return [normalize_text(x) for x in re.split(r"[|;,]\s*|\s{2,}", t) if x]

def extract_items(order_text) -> list:
    items = [x for x in split_items(order_text) if x]
    # unique but keep first occurrence order
    seen=set(); out=[]
    for it in items:
        if it not in seen:
            seen.add(it); out.append(it)
    return out

def row_iter(path, expected_cols=EXPECTED_COLS):
    exp_n = len(expected_cols)
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        _ = f.readline()  # header
        for line in f:
            line = line.rstrip("\n")
            if not line:
                continue
            parts = line.split(",", exp_n - 1)
            if len(parts) < exp_n:
                continue
            yield dict(zip(expected_cols, parts[:exp_n]))

# -----------------------------
# If you already have these maps from earlier cells, we reuse them.
# Otherwise we build minimally here.
# -----------------------------
need_build = False
for var in ["covis_global","covis_by_channel","covis_by_subch",
            "covis_by_occ","covis_by_store","covis_by_custtype",
            "pop_global","global_top","cart_size_hist"]:
    if var not in globals():
        need_build = True
        break

if need_build:
    covis_global      = defaultdict(Counter)
    covis_by_channel  = defaultdict(lambda: defaultdict(Counter))
    covis_by_subch    = defaultdict(lambda: defaultdict(Counter))
    covis_by_occ      = defaultdict(lambda: defaultdict(Counter))
    covis_by_store    = defaultdict(lambda: defaultdict(Counter))
    covis_by_custtype = defaultdict(lambda: defaultdict(Counter))
    pop_global        = Counter()
    cart_size_hist    = Counter()

    print("Building co-vis maps (quick build)...")
    for r in tqdm(row_iter(ORDER_PATH), unit="it"):
        items = extract_items(r.get("ORDERS",""))
        cart_size_hist[len(items)] += 1
        if not items:
            continue
        for it in set(items): pop_global[it]+=1
        uniq = list(set(items))
        n=len(uniq)
        if n>1:
            ch=r.get("ORDER_CHANNEL_NAME",""); sc=r.get("ORDER_SUBCHANNEL_NAME","")
            oc=r.get("ORDER_OCCASION_NAME",""); st=r.get("STORE_NUMBER","")
            ct=r.get("CUSTOMER_TYPE","")
            for i in range(n):
                a=uniq[i]
                for j in range(i+1,n):
                    b=uniq[j]
                    covis_global[a][b]+=1; covis_global[b][a]+=1
                    covis_by_channel[ch][a][b]+=1;  covis_by_channel[ch][b][a]+=1
                    covis_by_subch[sc][a][b]+=1;    covis_by_subch[sc][b][a]+=1
                    covis_by_occ[oc][a][b]+=1;      covis_by_occ[oc][b][a]+=1
                    covis_by_store[st][a][b]+=1;    covis_by_store[st][b][a]+=1
                    covis_by_custtype[ct][a][b]+=1; covis_by_custtype[ct][b][a]+=1
    global_top = [it for it,_ in pop_global.most_common(2000)]
else:
    print("Reusing previously built maps.")

# Normalize (keeps top neighbors; improves stability)
def normalize_covis(cmap: dict, pop: Counter, power=0.5, keep_top=500):
    out = {}
    for a, neigh in cmap.items():
        pa = max(pop[a], 1)
        denom_a = pa**power
        d = {}
        for b, c in neigh.items():
            pb = max(pop[b], 1)
            d[b] = float(c) / (denom_a * (pb**power))
        out[a] = dict(Counter(d).most_common(keep_top))
    return out

def norm_nested(ctx_map, pop):
    out = {}
    for key, cmap in ctx_map.items():
        out[key] = normalize_covis(cmap, pop, power=0.5)
    return out

covis_global_n      = normalize_covis(covis_global, pop_global, power=0.5)
covis_by_channel_n  = norm_nested(covis_by_channel,  pop_global)
covis_by_subch_n    = norm_nested(covis_by_subch,    pop_global)
covis_by_occ_n      = norm_nested(covis_by_occ,      pop_global)
covis_by_store_n    = norm_nested(covis_by_store,    pop_global)
covis_by_custtype_n = norm_nested(covis_by_custtype, pop_global)

# Tuned weights from your best run
W = (0.51, 0.24, 0.05, 0.11, 0.17, 0.10)   # (global, channel, subchannel, occasion, store, customer_type)
LAM = 0.93
BACKOFF_ALPHA = 0.15
CAND_POOL = 120

def blended_scores(cart, row):
    scores = Counter()
    def add_from(map_, weight):
        if weight <= 0 or not map_: return
        for it in cart:
            for c, v in map_.get(it, {}).items():
                scores[c] += weight * v

    ch = str(row.get("ORDER_CHANNEL_NAME",""))
    sc = str(row.get("ORDER_SUBCHANNEL_NAME",""))
    oc = str(row.get("ORDER_OCCASION_NAME",""))
    st = str(row.get("STORE_NUMBER",""))
    ct = str(row.get("CUSTOMER_TYPE",""))

    add_from(covis_global_n, W[0])
    if ch in covis_by_channel_n:  add_from(covis_by_channel_n[ch],  W[1])
    if sc in covis_by_subch_n:    add_from(covis_by_subch_n[sc],    W[2])
    if oc in covis_by_occ_n:      add_from(covis_by_occ_n[oc],      W[3])
    if st in covis_by_store_n:    add_from(covis_by_store_n[st],    W[4])
    if ct in covis_by_custtype_n: add_from(covis_by_custtype_n[ct], W[5])

    for it in cart: scores.pop(it, None)

    # backoff for sparse contexts
    if len(scores) < 5:
        for it in cart:
            for c, v in covis_global_n.get(it, {}).items():
                scores[c] += BACKOFF_ALPHA * v
        for it in cart: scores.pop(it, None)

    return scores

def recommend_simple(cart, row, k=3):
    scores = blended_scores(cart, row)
    cands = [c for c,_ in scores.most_common(800) if c not in cart][:CAND_POOL]
    # backfill popular if needed
    for g in global_top:
        if len(cands) >= CAND_POOL: break
        if g not in cart and g not in cands: cands.append(g)
    final = cands[:k]
    return (final + ["","",""])[:k], scores

# -----------------------------
# Strict LOO + Temporal metrics
# -----------------------------
def eval_strict_loo(n_eval=5000):
    hits1=hits2=hits3=map3=ndcg3=0.0; seen=0
    for r in row_iter(ORDER_PATH):
        items = extract_items(r.get("ORDERS",""))
        uniq = list(dict.fromkeys(items))
        if len(uniq) < 2: continue
        tgt = uniq[-1]
        cart = [x for x in uniq if x!=tgt]
        if not cart: continue
        preds,_ = recommend_simple(cart, r, k=3)
        if preds[0]==tgt: hits1+=1
        if tgt in preds[:2]: hits2+=1
        if tgt in preds[:3]: hits3+=1
        if tgt in preds[:3]:
            rank = preds.index(tgt)+1
            map3 += 1.0/rank
            ndcg3+= 1.0/math.log2(rank+1)
        seen += 1
        if seen>=n_eval: break
    return {
        "num_eval_orders": seen,
        "Recall@1": round(hits1/seen,4),
        "Recall@2": round(hits2/seen,4),
        "Recall@3": round(hits3/seen,4),
        "MAP@3": round(map3/seen,4),
        "NDCG@3": round(ndcg3/seen,4)
    }

def count_rows_fast(path):  # for temporal split location
    return max(0, sum(1 for _ in open(path, "r", encoding="utf-8", errors="replace")) - 1)

def eval_temporal_tail(tail_frac=0.1, max_eval=8000):
    total = count_rows_fast(ORDER_PATH)
    train_cut = int(total*(1-tail_frac))
    hits1=hits2=hits3=map3=ndcg3=0.0; seen=0; i=0
    for r in row_iter(ORDER_PATH):
        i+=1
        if i<=train_cut:
            continue
        items = extract_items(r.get("ORDERS",""))
        uniq = list(dict.fromkeys(items))
        if len(uniq) < 2: continue
        tgt = uniq[-1]
        cart = [x for x in uniq if x!=tgt]
        if not cart: continue
        preds,_ = recommend_simple(cart, r, k=3)
        if preds[0]==tgt: hits1+=1
        if tgt in preds[:2]: hits2+=1
        if tgt in preds[:3]: hits3+=1
        if tgt in preds[:3]:
            rank = preds.index(tgt)+1
            map3 += 1.0/rank
            ndcg3+= 1.0/math.log2(rank+1)
        seen += 1
        if seen>=max_eval: break
    return {
        "num_eval_orders": seen,
        "Recall@1": round(hits1/seen,4),
        "Recall@2": round(hits2/seen,4),
        "Recall@3": round(hits3/seen,4),
        "MAP@3": round(map3/seen,4),
        "NDCG@3": round(ndcg3/seen,4)
    }

strict_m = eval_strict_loo(n_eval=5000)
temp_m   = eval_temporal_tail(tail_frac=0.1, max_eval=8000)
print("Strict LOO:", strict_m)
print("Temporal:  ", temp_m)

# Save metrics
with open(os.path.join(OUT_DIR, "metrics.json"), "w") as f:
    json.dump({"strict_looo": strict_m, "temporal": temp_m}, f, indent=2)
with open(os.path.join(OUT_DIR, "metrics.csv"), "w") as f:
    f.write("Eval,Num,Recall@1,Recall@2,Recall@3,MAP@3,NDCG@3\n")
    f.write("Strict LOO,{num},{r1},{r2},{r3},{map3},{ndcg3}\n".format(
        num=strict_m["num_eval_orders"], r1=strict_m["Recall@1"], r2=strict_m["Recall@2"],
        r3=strict_m["Recall@3"], map3=strict_m["MAP@3"], ndcg3=strict_m["NDCG@3"]))
    f.write("Temporal,{num},{r1},{r2},{r3},{map3},{ndcg3}\n".format(
        num=temp_m["num_eval_orders"], r1=temp_m["Recall@1"], r2=temp_m["Recall@2"],
        r3=temp_m["Recall@3"], map3=temp_m["MAP@3"], ndcg3=temp_m["NDCG@3"]))

# -----------------------------
# Charts: long tail, top-20, cart size, heatmap, recall, calibration, lift
# -----------------------------
# Long tail
freqs = [v for _,v in pop_global.most_common()]
plt.figure(figsize=(7,4.5)); plt.plot(range(1,len(freqs)+1), freqs)
plt.xlabel("Items sorted by popularity rank"); plt.ylabel("Orders count")
plt.title("Item Popularity Long Tail"); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "long_tail.png"), dpi=160); plt.close()

# Top-20
top20 = pop_global.most_common(20)
plt.figure(figsize=(8,5)); plt.bar([k for k,_ in top20], [v for _,v in top20])
plt.xticks(rotation=60, ha="right"); plt.ylabel("Orders count")
plt.title("Top 20 Items"); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "top20.png"), dpi=160); plt.close()

# Cart size histogram
xs = sorted(dict(cart_size_hist).items())
plt.figure(figsize=(7,4.5)); plt.bar([k for k,_ in xs], [v for _,v in xs])
plt.xlabel("Cart size"); plt.ylabel("Orders")
plt.title("Cart Size Distribution"); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "cart_size_hist.png"), dpi=160); plt.close()

# Co-vis heatmap (Top-20, normalized)
import numpy as np
top20_items = [k for k,_ in top20][:20]
M = np.zeros((len(top20_items), len(top20_items)), dtype=float)
for i,a in enumerate(top20_items):
    neigh = covis_global.get(a, {})
    for j,b in enumerate(top20_items):
        if a==b: continue
        M[i,j] = neigh.get(b, 0.0)
if M.max() > 0:
    M = M / (M.max()+1e-9)
plt.figure(figsize=(7,6)); plt.imshow(M, aspect="auto")
plt.xticks(range(len(top20_items)), top20_items, rotation=60, ha="right")
plt.yticks(range(len(top20_items)), top20_items)
plt.title("Normalized Co-Visitation Heatmap (Top-20)"); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "covis_heatmap.png"), dpi=160); plt.close()

# Recall@k bar chart (Strict LOO vs Temporal)
labels = ["R@1","R@2","R@3"]
vals_m = [strict_m["Recall@1"], strict_m["Recall@2"], strict_m["Recall@3"]]
vals_t = [temp_m["Recall@1"],   temp_m["Recall@2"],   temp_m["Recall@3"]]
x = np.arange(3)
plt.figure(figsize=(6.5,4.5))
plt.bar(x-0.15, vals_m, width=0.3, label="Strict LOO")
plt.bar(x+0.15, vals_t, width=0.3, label="Temporal")
plt.xticks(x, labels); plt.ylim(0,1); plt.legend()
plt.title("Recall@k — Strict LOO vs Temporal")
plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, "recall_chart.png"), dpi=160); plt.close()

# Calibration (top-1 confidence proxy = softmax of top-3 raw scores)
def softmax(x):
    x = np.array(x, dtype=np.float32)
    x = x - x.max()
    e = np.exp(x)
    s = e.sum() + 1e-12
    return (e / s).tolist()

confs=[]; hits=[]
i=0
for r in row_iter(ORDER_PATH):
    items = extract_items(r.get("ORDERS",""))
    uniq = list(dict.fromkeys(items))
    if len(uniq)<2: continue
    tgt = uniq[-1]; cart = [x for x in uniq if x!=tgt]
    if not cart: continue
    preds, scores = recommend_simple(cart, r, k=3)
    raw = [scores.get(p, 1.0/(j+1)) for j,p in enumerate(preds[:3])]
    c = softmax(raw)[0] if raw else 0.33
    confs.append(c)
    hits.append(1.0 if (len(preds)>0 and preds[0]==tgt) else 0.0)
    i+=1
    if i>=4000: break

confs = np.array(confs); hits = np.array(hits)
dec = np.minimum((confs*10).astype(int), 9)
xb, yb = [], []
for d in range(10):
    m = (dec==d)
    xb.append(d/10.0); yb.append(float(hits[m].mean()) if m.sum()>0 else 0.0)

plt.figure(figsize=(6.5,4.5))
plt.plot(xb, yb)
plt.plot([0,0.9],[xb[0],xb[-1]], linestyle="--")
plt.xlabel("Confidence decile (lower bound)"); plt.ylabel("Empirical Hit-Rate (Top-1)")
plt.title("Confidence Calibration (Top-1)")
plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, "confidence_calibration.png"), dpi=160); plt.close()

# Lift curve
order = np.argsort(-confs)
cum_hits = np.cumsum(hits[order])
pct = np.arange(1, len(confs)+1) / len(confs)
lift = cum_hits / (np.arange(1,len(confs)+1) * hits.mean() + 1e-9)
plt.figure(figsize=(6.5,4.5)); plt.plot(pct, lift)
plt.xlabel("Fraction of recommendations shown"); plt.ylabel("Lift vs random")
plt.title("Lift Curve (Top-1 by confidence)")
plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, "lift_curve.png"), dpi=160); plt.close()

# Context weights (if you don't have a model FI)
plt.figure(figsize=(6.5,4.5))
plt.bar(range(len(W)), list(W))
plt.xticks(range(len(W)), ["Global","Channel","Subchannel","Occasion","Store","CustomerType"])
plt.ylim(0, max(W)+0.1)
plt.title("Context Contribution Weights (Tuned)")
plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, "context_weights.png"), dpi=160); plt.close()

print("Saved graphs + metrics in:", OUT_DIR)


Building co-vis maps (quick build)...


1070135it [06:10, 2889.44it/s]


Strict LOO: {'num_eval_orders': 5000, 'Recall@1': 0.8716, 'Recall@2': 0.8738, 'Recall@3': 0.874, 'MAP@3': 0.8728, 'NDCG@3': 0.8731}
Temporal:   {'num_eval_orders': 8000, 'Recall@1': 0.8498, 'Recall@2': 0.8515, 'Recall@3': 0.8525, 'MAP@3': 0.851, 'NDCG@3': 0.8514}
Saved graphs + metrics in: /content


In [ ]:
# ==== Micro-tuner for context-blended co-vis recommender ====
# Tries small grids for (Channel, Store, Occasion weights), CAND_POOL, and BACKOFF_ALPHA.
# Produces: tuning_report.csv, tuning_plot.png, best_config.json
# Optional: rewrites Recommendation_Output_MAX.xlsx using the best config.

import os, re, csv, json, math, random, time
from collections import defaultdict, Counter
from statistics import mean

try:
    import pandas as pd
    PANDAS_OK = True
except Exception:
    PANDAS_OK = False

import matplotlib.pyplot as plt

# --------- Paths (edit if needed) ----------
ORDER_PATH = os.getenv("ORDER_PATH", "/content/order_data.csv")
TEST_PATH  = os.getenv("TEST_PATH",  "/content/test_data_question.csv")
OUT_DIR    = os.getenv("OUT_DIR",    "/content")
os.makedirs(OUT_DIR, exist_ok=True)

# --------- Quick controls ----------
MAX_ROWS_BUILD   = int(os.getenv("MAX_ROWS_BUILD", "300000"))  # subset for fast tuning
N_EVAL           = int(os.getenv("N_EVAL", "3000"))            # strict-LOO eval sample
WRITE_FINAL_XLSX = os.getenv("WRITE_FINAL_XLSX", "false").lower() in {"1","true","yes"}

# --------- Robust line reader (preserves commas in last column) ----------
EXPECTED = ("CUSTOMER_ID","ORDER_ID","ORDER_CHANNEL_NAME","ORDER_SUBCHANNEL_NAME",
            "ORDER_OCCASION_NAME","STORE_NUMBER","CUSTOMER_TYPE","ORDERS")
EXP_N = len(EXPECTED)

def row_iter(path, max_rows=None):
    n = 0
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        _ = f.readline()  # header (ignored)
        for line in f:
            line = line.rstrip("\n")
            if not line:
                continue
            parts = line.split(",", EXP_N - 1)
            if len(parts) < EXP_N:
                continue
            row = dict(zip(EXPECTED, parts[:EXP_N]))
            yield row
            n += 1
            if max_rows and n >= max_rows:
                break

def extract_items(s):
    if s is None: return []
    s = str(s).strip().strip('"').strip("'")
    if not s: return []
    parts = re.split(r'[|,;]\s*', s)
    out=[]; seen=set()
    for p in parts:
        p=p.strip()
        if not p or p.lower() in {"na","none","null","nan"}:
            continue
        if p not in seen:
            seen.add(p); out.append(p)
    return out

# --------- Build co-vis maps (subset for speed) ----------
def build_covis(limit_rows=None):
    covis_global      = defaultdict(Counter)
    covis_by_channel  = defaultdict(lambda: defaultdict(Counter))
    covis_by_subch    = defaultdict(lambda: defaultdict(Counter))
    covis_by_occ      = defaultdict(lambda: defaultdict(Counter))
    covis_by_store    = defaultdict(lambda: defaultdict(Counter))
    covis_by_custtype = defaultdict(lambda: defaultdict(Counter))
    pop = Counter()

    t0=time.time(); rows=0; orders=0
    for r in row_iter(ORDER_PATH, max_rows=limit_rows):
        items = extract_items(r.get("ORDERS",""))
        if not items:
            rows += 1;
            continue
        orders += 1; rows += 1
        uniq = list(dict.fromkeys(items))
        for it in uniq: pop[it]+=1
        L=len(uniq)
        if L>1:
            ch=r.get("ORDER_CHANNEL_NAME",""); sc=r.get("ORDER_SUBCHANNEL_NAME","")
            oc=r.get("ORDER_OCCASION_NAME",""); st=r.get("STORE_NUMBER","")
            ct=r.get("CUSTOMER_TYPE","")
            for i in range(L):
                a=uniq[i]
                for j in range(i+1, L):
                    b=uniq[j]
                    covis_global[a][b]+=1; covis_global[b][a]+=1
                    covis_by_channel[ch][a][b]+=1;  covis_by_channel[ch][b][a]+=1
                    covis_by_subch[sc][a][b]+=1;    covis_by_subch[sc][b][a]+=1
                    covis_by_occ[oc][a][b]+=1;      covis_by_occ[oc][b][a]+=1
                    covis_by_store[st][a][b]+=1;    covis_by_store[st][b][a]+=1
                    covis_by_custtype[ct][a][b]+=1; covis_by_custtype[ct][b][a]+=1

        if rows % 200_000 == 0:
            print(f"Processed {rows:,} rows...")

    gtop = [it for it,_ in pop.most_common(1500)]
    t1=time.time()
    print(f"Build (subset) — Rows: {rows:,} | Orders: {orders:,} | Unique items: {len(pop)} | {(t1-t0)/60:.2f} min")
    return (covis_global, covis_by_channel, covis_by_subch,
            covis_by_occ, covis_by_store, covis_by_custtype, pop, gtop)

(cg, by_ch, by_sc, by_oc, by_st, by_ct, POP, GLOBAL_TOP) = build_covis(MAX_ROWS_BUILD)

# --------- Blended scorer with tunables ----------
DEFAULT_W = (0.51, 0.24, 0.05, 0.11, 0.17, 0.10)  # (global, channel, subch, occasion, store, custtype)

def blended_scores(cart, row, W, CAND_POOL=120, BACKFILL_ALPHA=0.15):
    sc = Counter()
    def add_from(map_, weight):
        if weight <= 0: return
        for it in cart:
            for c, v in map_.get(it, {}).items():
                sc[c] += weight * v

    # base/global
    add_from(cg, W[0])

    ch = row.get("ORDER_CHANNEL_NAME","")
    sub= row.get("ORDER_SUBCHANNEL_NAME","")
    oc = row.get("ORDER_OCCASION_NAME","")
    st = row.get("STORE_NUMBER","")
    ct = row.get("CUSTOMER_TYPE","")

    if ch in by_ch:  add_from(by_ch[ch],  W[1])
    if sub in by_sc: add_from(by_sc[sub], W[2])
    if oc in by_oc:  add_from(by_oc[oc],  W[3])
    if st in by_st:  add_from(by_st[st],  W[4])
    if ct in by_ct:  add_from(by_ct[ct],  W[5])

    for it in cart: sc.pop(it, None)

    if len(sc) < 5:   # sparse backoff
        for it in cart:
            for c, v in cg.get(it, {}).items():
                sc[c] += BACKFILL_ALPHA * v
        for it in cart: sc.pop(it, None)

    # return top candidates (list)
    cands = [c for c,_ in sc.most_common(max(CAND_POOL, 3)) if c not in cart]
    # backfill with global top
    for g in GLOBAL_TOP:
        if len(cands) >= max(CAND_POOL,3): break
        if g not in cart and g not in cands:
            cands.append(g)
    return sc, cands

def recommend(cart, row, W, CAND_POOL=120, BACKFILL_ALPHA=0.15, k=3):
    scores, cands = blended_scores(cart, row, W, CAND_POOL, BACKFILL_ALPHA)
    return (cands[:k] + ["","",""])[:k]

# --------- Strict LOO eval ----------
def eval_strict_looo(W, CAND_POOL=120, BACKFILL_ALPHA=0.15, n_eval=N_EVAL, seed=42):
    random.seed(seed)
    hits1=[]; hits2=[]; hits3=[]; map3=[]; ndcg3=[]
    seen=0
    for row in row_iter(ORDER_PATH, max_rows=None):
        items = extract_items(row.get("ORDERS",""))
        uniq  = list(dict.fromkeys(items))
        if len(uniq) < 2:
            continue
        target = uniq[-1]
        cart   = [x for x in uniq if x != target]
        if not cart:
            continue
        R = recommend(cart, row, W, CAND_POOL=CAND_POOL, BACKFILL_ALPHA=BACKOFF_ALPHA)
        hits1.append(1.0 if target == R[0] else 0.0)
        hits2.append(1.0 if target in R[:2] else 0.0)
        hits3.append(1.0 if target in R[:3] else 0.0)
        # MAP@3
        ap=0.0
        for i, r in enumerate(R[:3], start=1):
            if r == target: ap = 1.0/i; break
        map3.append(ap)
        # NDCG@3
        dcg = 0.0
        for i, r in enumerate(R[:3], start=1):
            rel = 1.0 if r == target else 0.0
            if rel>0:
                dcg += (2**rel - 1)/math.log2(i+1)
        ndcg3.append(dcg/1.0)
        seen += 1
        if seen >= n_eval:
            break
    return {
        "num_eval": seen,
        "R1": round(mean(hits1), 5),
        "R2": round(mean(hits2), 5),
        "R3": round(mean(hits3), 5),
        "MAP3": round(mean(map3), 5),
        "NDCG3": round(mean(ndcg3), 5),
    }

# --------- Small grid around best-known values ----------
GRID_CHANNEL  = [0.20, 0.24, 0.28]
GRID_STORE    = [0.15, 0.18, 0.21]
GRID_OCCASION = [0.08, 0.11, 0.14]
GRID_CANDPOOL = [100, 120, 160]
GRID_BACKOFF  = [0.10, 0.15, 0.20]

trials = []
t0=time.time()
trial_id=0
for ch in GRID_CHANNEL:
    for st in GRID_STORE:
        for oc in GRID_OCCASION:
            for cp in GRID_CANDPOOL:
                for bk in GRID_BACKOFF:
                    W = (0.51, ch, 0.05, oc, st, 0.10)  # keep subch & custtype stable
                    BACKOFF_ALPHA = bk
                    res = eval_strict_looo(W, CAND_POOL=cp, BACKFILL_ALPHA=BACKOFF_ALPHA, n_eval=N_EVAL)
                    trial_id += 1
                    trials.append({
                        "trial": trial_id,
                        "W": W,
                        "CAND_POOL": cp,
                        "BACKOFF_ALPHA": BACKOFF_ALPHA,
                        **res
                    })
                    print(f"[{trial_id:03d}] W={W} CP={cp} BK={bk} -> R@3={res['R3']:.5f}")

print(f"Tuning done in {(time.time()-t0)/60:.2f} min | trials={len(trials)}")

# --------- Save report & plot ----------
trials_sorted = sorted(trials, key=lambda x: (x["R3"], x["MAP3"], x["R1"]), reverse=True)
report_csv = os.path.join(OUT_DIR, "tuning_report.csv")
with open(report_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=list(trials_sorted[0].keys()))
    w.writeheader()
    for row in trials_sorted: w.writerow(row)

best = trials_sorted[0]
with open(os.path.join(OUT_DIR, "best_config.json"), "w") as f:
    json.dump(best, f, indent=2)

# simple plot
plt.figure(figsize=(8,4.5))
plt.plot([t["trial"] for t in trials_sorted], [t["R3"] for t in trials_sorted])
plt.xlabel("Trial (sorted)")
plt.ylabel("Recall@3")
plt.title("Micro-tune: Recall@3 across trials")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "tuning_plot.png"), dpi=160); plt.close()

print("Best config:", best)
print("Saved:", report_csv, "and tuning_plot.png & best_config.json in", OUT_DIR)

# --------- Optional: regenerate competition sheet with best config ----------
if WRITE_FINAL_XLSX and PANDAS_OK:
    print("Regenerating Recommendation_Output_MAX.xlsx with tuned config (subset-built maps)...")
    df = pd.read_csv(TEST_PATH, dtype=str, keep_default_na=False)
    item_cols = [c for c in df.columns if c.upper().startswith("ITEM")]
    p1,p2,p3=[],[],[]
    for _, r in df.iterrows():
        cart = [str(r[c]).strip() for c in item_cols if r.get(c,"")]
        recs = recommend(cart, r, tuple(best["W"]), CAND_POOL=int(best["CAND_POOL"]),
                         BACKFILL_ALPHA=float(best["BACKOFF_ALPHA"]), k=3)
        p1.append(recs[0]); p2.append(recs[1]); p3.append(recs[2])
    out=df.copy()
    out["RECOMMENDATION 1"]=p1; out["RECOMMENDATION 2"]=p2; out["RECOMMENDATION 3"]=p3
    cols = ["CUSTOMER_ID","ORDER_ID"] + item_cols + ["RECOMMENDATION 1","RECOMMENDATION 2","RECOMMENDATION 3"]
    out[cols].to_excel(os.path.join(OUT_DIR, "Recommendation_Output_MAX.xlsx"), index=False)
    print("Wrote:", os.path.join(OUT_DIR, "Recommendation_Output_MAX.xlsx"),
          "(note: tuned on subset-built maps — re-run on full data for final lock-in)")


Processed 200,000 rows...
Build (subset) — Rows: 300,000 | Orders: 300,000 | Unique items: 3171 | 1.24 min
[001] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=100 BK=0.1 -> R@3=0.87067
[002] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=100 BK=0.15 -> R@3=0.87067
[003] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=100 BK=0.2 -> R@3=0.87067
[004] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=120 BK=0.1 -> R@3=0.87067
[005] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=120 BK=0.15 -> R@3=0.87067
[006] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=120 BK=0.2 -> R@3=0.87067
[007] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=160 BK=0.1 -> R@3=0.87067
[008] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=160 BK=0.15 -> R@3=0.87067
[009] W=(0.51, 0.2, 0.05, 0.08, 0.15, 0.1) CP=160 BK=0.2 -> R@3=0.87067
[010] W=(0.51, 0.2, 0.05, 0.11, 0.15, 0.1) CP=100 BK=0.1 -> R@3=0.87067
[011] W=(0.51, 0.2, 0.05, 0.11, 0.15, 0.1) CP=100 BK=0.15 -> R@3=0.87067
[012] W=(0.51, 0.2, 0.05, 0.11, 0.15, 0.1) CP=100 BK=0.2 -> R@3=0.87067
[013] W=(0.51, 0.2, 0.05,